provision-test longevity for interactive play with sct cluster

In [ ]:
import os
os.environ["BUILD_USER_EMAIL"] = "sct.tester@scylladb.com"
os.environ["BUILD_ID"] = "123"
os.environ["SCT_CLUSTER_BACKEND"] = "aws"
os.environ["SCT_CONFIG_FILES"] = "test-cases/PR-provision-test.yaml"
os.environ["SCT_SCYLLA_VERSION"] = "2022.2.0"
os.environ["SCT_REGION_NAME"] = "eu-west-1"
os.environ["SCT_USE_MGMT"] = "False"
os.environ["SCT_N_DB_NODES"] = "3"
os.environ["SCT_N_LOADERS"] = "1"
os.environ["SCT_N_MONITORS_NODES"] = "0"
assert os.environ["BUILD_USER_EMAIL"] != "sct.tester@scylladb.com", "please use your own email so resources are tracked properly"

# logging configuration, for jupyter only (sct logs are intact)
import logging

LOGGER = logging.getLogger(__name__)
LOGGER.setLevel(logging.DEBUG)
consoleHandler = logging.StreamHandler()
consoleHandler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
consoleHandler.setFormatter(formatter)
LOGGER.addHandler(consoleHandler)

In [ ]:
from longevity_test import LongevityTest
import os

from sdcm import sct_abs_path

os.chdir(sct_abs_path(relative_filename=""))
tester_inst = LongevityTest()
tester_inst.setUpClass()
tester_inst.setUp()
cluster = tester_inst.db_cluster


Now it is possible to play interactively with cluster. E.g.:

```
res = cluster.nodes[0].remoter.run("nodetool status")
res.stdout
```
Remember, imported code can't be reimported - when changed, need to reload jupyter kernel.
A way to workaround it, use inheritance and override methods. E.g.:
```
from sdcm.nemesis import Nemesis


class DemoNemesis(Nemesis):
    disruptive = True
    networking = False
    run_with_gemini = False

    def disrupt(self):
        self._disrupt_demo()

    def _disrupt_demo(self):
        print("starting demo nemesis")

    def update_stats(self, disrupt, status=True, data=None):
        if not data:
            data = {}
        key = {True: 'runs', False: 'failures'}
        if disrupt not in self.stats:
            self.stats[disrupt] = {'runs': [], 'failures': [], 'cnt': 0}
        self.stats[disrupt][key[status]].append(data)
        self.stats[disrupt]['cnt'] += 1
        self.log.debug('Update nemesis info with: %s', data)
        if self.tester.create_stats:
            self.tester.update({'nemesis': self.stats})
        if self.es_publisher:
            self.es_publisher.publish(disrupt_name=disrupt, status=status, data=data)

demo_nemesis = DemoNemesis(tester_inst, termination_event=None)
demo_nemesis.disrupt()
```


To be able to play cluster object methods interactively, just create function that uses `self` as first argument and assign `cluster` to it. When having working example, copy-paste method to cluster with proper indent. E.g.:
```
from typing import List


def get_non_system_ks_cf_list(self, db_node, filter_out_table_with_counter=False, filter_out_mv=False, filter_empty_tables=True) -> List[str]:
    ks_cf_list = self.get_any_ks_cf_list(db_node, filter_out_table_with_counter=filter_out_table_with_counter,
                                       filter_out_mv=filter_out_mv, filter_empty_tables=filter_empty_tables,
                                       filter_out_system=True, filter_out_cdc_log_tables=True)
    return ["some_non_system_table"] + ks_cf_list

get_non_system_ks_cf_list(cluster, cluster.nodes[0])
```